#### this is an experimental script to test an hypothesis.

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_pascal_voc(
    images_directory_path=f"LLVIP/infrared/train",
    annotations_directory_path=f"LLVIP/Annotations"
)

train_ds, test_ds = ds.split(split_ratio=0.7,
                             random_state=42, shuffle=True)
len(train_ds), len(test_ds)

In [ ]:
import os
# Define directories for your YOLO dataset
yolo_dataset_dir = "LLVIP_YOLO_Dataset_Infrared"
train_images_dir = os.path.join(yolo_dataset_dir, "images", "train")
train_labels_dir = os.path.join(yolo_dataset_dir, "labels", "train")
test_images_dir = os.path.join(yolo_dataset_dir, "images", "test")
test_labels_dir = os.path.join(yolo_dataset_dir, "labels", "test")

In [ ]:
# Create directories
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)


In [ ]:
# Export train_ds
# Call as_yolo directly on the train_ds object
train_ds.as_yolo(
    images_directory_path=train_images_dir,
    annotations_directory_path=train_labels_dir,
    min_image_area_percentage=0.0,
    max_image_area_percentage=1.0,
    # The class_map will be derived from train_ds.classes automatically
)

In [ ]:
# Export test_ds
# Call as_yolo directly on the test_ds object
test_ds.as_yolo(
    images_directory_path=test_images_dir,
    annotations_directory_path=test_labels_dir,
    min_image_area_percentage=0.0,
    max_image_area_percentage=1.0,
)

In [ ]:
# Create a data.yaml file (crucial for YOLOv8)
# This file tells YOLOv8 where your data is and what classes you have.
data_yaml_path = os.path.join(yolo_dataset_dir, "data.yaml")
with open(data_yaml_path, 'w') as f:
    f.write(f"path: {os.path.abspath(yolo_dataset_dir)}\n")
    f.write("train: images/train\n")
    f.write("val: images/test\n") # YOLOv8 uses 'val' for validation/testing
    f.write(f"nc: {len(train_ds.classes)}\n") # Number of classes
    f.write("names:\n")
    for i, class_name in enumerate(train_ds.classes):
        f.write(f"  {i}: {class_name}\n")

print(f"YOLO dataset prepared at: {yolo_dataset_dir}")
print(f"Data YAML file created at: {data_yaml_path}")

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8n (nano) model. You can choose other models like 'yolov8s.pt' (small), 'yolov8m.pt' (medium), etc.
# 'n' is the smallest and fastest, good for quick testing.
model = YOLO('yolov8n.pt')

# Train the model

results = model.train(data='LLVIP_YOLO_Dataset/data.yaml', epochs=100, imgsz=640, batch=16, name='LLVIP_YOLOv8_model', freeze=5)

# The training results (weights, logs, etc.) will be saved in a directory like 'runs/detect/LLVIP_YOLOv8_modelX'

In [ ]:
from ultralytics import YOLO
# Load the best trained model weights
model_path = 'runs/detect/LLVIP_YOLOv8_model4/weights/best.pt' # Adjust this path
model = YOLO(model_path)

# Evaluate the model on the validation/test set
# 'data' points to your data.yaml
metrics = model.val(data='LLVIP_YOLO_Dataset_Infrared/data.yaml')

# Print metrics
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")
print(f"Precision: {metrics.box.p}")
print(f"Recall: {metrics.box.r}")